#  Reservoir Routing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma

In [2]:
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

In [3]:
#Libraries for linear regression 
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.metrics import r2_score # measure the model performance

import piecewise_regression #Piecewise linear regression

In [4]:
import rasterio

#Projecting the files
import cartopy.crs as ccrs  # projection

import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon

In [5]:
# Cliping netcdf file
from shapely.geometry import mapping
import geopandas as gpd  #for reading the shapefile


In [6]:
from scipy import optimize
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [11]:
from datetime import date
from datetime import timedelta

In [ ]:
# Observed sub-basin discharges 
df_q= pd.read_excel('path_to_file', index_col=0,
                   parse_dates= True)


#Simulated subcatchment discharge
df_subcat= pd.read_excel('path_to_file', index_col=0, parse_dates=True)


## Natural conditions Q= f(H)
**Input

Storage- height (Depth) curve

Discharge Q- Height(depth) curve

Iniitial reservoir condition i.e Water level

Method: Mass balance

In [28]:
# alogirthms for reservoir operation
def linear_regression(x, y):
    slope, intercept, r, p, std_err = stats.linregress(x, y)
    def myfunc(x):
        return slope*x + intercept
    y_pred = np.array(list(map(myfunc, x)))    
    return y_pred, r, myfunc



def poly_regression(x, y, degree):
    model= np.poly1d(np.polyfit(x, y, degree))
    
    return model

def piecewise_linear_reg(x, y): # For one break boint
    def piecewise_linear(x, x0, y0, k1, k2):
        return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])
    p , e = optimize.curve_fit(piecewise_linear, x, y)
    
    return piecewise_linear, p

def piece_fit_linreg(x, y, n): #Piecewise linear fit ofr n break points
    pw_fit = piecewise_regression.Fit(x, y, n_breakpoints=n)
    
    #y_pred= pw_fit.predict(x)
    
    return pw_fit
    


#Volume depth curve relationship
def volume_depth_Q(S, H, h, q, degree):
   
    #Making functions and precictions
    
    S_pred, r1, myfunc_SH= linear_regression(x=H, y=S) # s=f(H)
    H_pred, r2, myfunc_HS= linear_regression(x=S, y=H) # H= f(S)


    #Creating relation between national discharge q and height h
   
    #Making predictions
    q_h_model= poly_regression(h,  q, degree)


    #Finding G-Q relationship
    Q_H= q_h_model(h) # Discharge h is the discharge array , Q_h is discharge as a fuction of H 
    S_H = myfunc_SH(h) # Volume as a function of H

    # using a Polynominal regression 
    #Creating relation between G, Q, and S 
    G = S_H*10**6/86400 + Q_H/2

    #Finding G-Q relationship
    # using a Polynominal regression 
    G_Q_model= np.poly1d(np.polyfit(G, Q_H, 4))
    #line_G = np.linspace(np.min(G), np.max(G), len(G))
    
    
    return  myfunc_SH,r1,  myfunc_HS, r2, G_Q_model, q_h_model



def reservoir_routing(I, h0, S_H, q_h, G_Q, H_S ): # h0 intitial water level,  S_H- storage as func(Height),
    #  q_h- dicharge as f(height), G_Q= G as func(Q), H_S- Height as func(Storage)
    #Creating arrays
    t= 24*60*60  # Time step in one day
    S= np.zeros(len(I)) #Volume
    G= np.zeros(len(I))
    Q= np.zeros(len(I))#outflow
    H= np.zeros(len(I))# Resevoir Height
    
    #Specifying initial conditions
    H[0]= h0  # meters
    S[0]= S_H(H[0]) # Storage as a function of height
    Q[0]= q_h(H[0])
    G[0]= S[0]*10**6/86400 + Q[0]/2
    
    
    #Resevoir routine 
    for i in range(1,len(I)):
        G[i]= G[i-1]+(I[i-1]+I[i])/2-Q[i-1]
        
        '''note to evapouration:''' 
        #G[i]= G[i-1]+((I[i-1]+I[i])-(E[i-1]+E[i]))/2-Q[i-1] ## Incoprating evapouration
        
        Q[i]= G_Q(G[i])  # Outflow  G_Q function

        S[i]=((G[i]-(Q[i]/2))*t)/10**6 # Storage

        H[i]= H_S(S[i])  # Height as function of storage
        
        
        
    #Convert to dataframe
    df_inflow= pd.DataFrame(I)
    df_outflow= pd.DataFrame(Q)
    df_storage= pd.DataFrame(S)
    df_head= pd.DataFrame(H)


    #Rename respectively date frame and add index
    df_inflow.columns=['Inflow (m3/s)']
    df_outflow.columns=['Outflow (m3/s)']
    df_storage.columns=['Storage (MCM)']
    df_head.columns=['Height (m)']

    #Join the data frames
    result = pd.concat([df_inflow,df_outflow,df_storage, df_head], axis=1, join='inner')
    result.index= df_routed.index
    
    return result



In [29]:
#Volume depth curve piecewise relationship
def pw_volume_depth_Q(S, H, h, q, degree, n1, n2):
    
    #S is the Storag and volume and H is the corresponding water level as numpy arrays
    
    #h and q is the historical timeseries as np.array 
   
    #Making functions and precictions
    
    myfunc_SH= piece_fit_linreg(x=H, y=S, n=n1) # s=f(H)
    myfunc_HS= piece_fit_linreg(x=S, y=H, n= n2) # H= f(S)


    #Creating relation between national discharge q and height h
   
    #Making predictions
    #q_h_model= poly_regression(h,  q, degree)
    q_h_model= piece_fit_linreg(h,  q, 2)
    
    
    #Finding G-Q relationship
    #Q_H= q_h_model(h) # Discharge h is the discharge array , Q_h is discharge as a fuction of H 
    Q_H= q_h_model.predict(h) # Discharge h is the discharge array , Q_h is discharge as a fuction of H
    S_H = myfunc_SH.predict(h) # Volume as a function of H

    # using a Polynominal regression 
    #Creating relation between G, Q, and S 
    G = S_H*10**6/86400 + Q_H/2

    #Finding G-Q relationship
    # using a Polynominal regression 
    G_Q_model= np.poly1d(np.polyfit(G, Q_H, 4))
    #line_G = np.linspace(np.min(G), np.max(G), len(G))
    
    
    return  myfunc_SH,  myfunc_HS,  G_Q_model, q_h_model

def pw_reservoir_routing(I, h0, S_H, q_h, G_Q, H_S): # h0 intitial water level,  S_H- storage as func(Height),
    #  q_h- dicharge as f(height), G_Q= G as func(Q), H_S- Height as func(Storage)
    #Creating arrays
    t= 24*60*60  # Time step in one day
    S= np.zeros(len(I)) #Volume
    G= np.zeros(len(I))
    Q= np.zeros(len(I))#outflow
    H= np.zeros(len(I))# Resevoir Height
    
    #Specifying initial conditions
    H[0]= h0  # meters
    S[0]= S_H.predict([H[0]]) # Storage as a function of height_ list
    Q[0]= q_h.predict([H[0]]) #List
    G[0]= S[0]*10**6/86400 + Q[0]/2
    
    
    #Resevoir routine 
    for i in range(1,len(I)):
        G[i]= G[i-1]+(I[i-1]+I[i])/2-Q[i-1]
        Q[i]= G_Q(G[i])  # Outflow  G_Q function

        S[i]=((G[i]-(Q[i]/2))*t)/10**6 # Storage

        H[i]= H_S.predict([S[i]])  # Height as function of storage #List
        
       
        
    #Convert to dataframe
    df_inflow= pd.DataFrame(I)
    df_outflow= pd.DataFrame(Q)
    df_storage= pd.DataFrame(S)
    df_head= pd.DataFrame(H)


    #Rename respectively date frame and add index
    df_inflow.columns=['Inflow (m3/s)']
    df_outflow.columns=['Outflow (m3/s)']
    df_storage.columns=['Storage (MCM)']
    df_head.columns=['Height (m)']

    #Join the data frames
    result = pd.concat([df_inflow,df_outflow,df_storage, df_head], axis=1, join='inner')
    result.index= df_routed.index
    
    return result

###  Developing Reservoir curves

In [33]:
#Stage (H)- volume (S) curves

S= np.array(storage_volume) # Reservoir storage
H= np.array(water_level) # Reservoir Height

#H_Q curves 

h, q = np.array(df['H (m)']), np.array(df['Q (m3/s)']) # historical time series
qmin= np.min(q)

#Using linear H_S curves
myfunc_SH, r1, myfunc_HS, r2, G_Q_model, q_h_model= volume_depth_Q(S, H, h, q, degree= 3) # polynominal degree. 
             
#Using piecewise linear regression of H_S curves
myfunc_SH2, myfunc_HS2, G_Q_model2, q_h_model2= pw_volume_depth_Q(S2, H2, h2, q2, degree=3, n1=2, n2=1)